# Writing out a USGSCSM ISD from a PDS3 Messenger MDIS image

In [1]:
import os 
os.environ["ALESPICEROOT"] = '/usgs/cpkgs/isis3/data'
import ale
from ale.drivers.mess_drivers import MessengerMdisPds3NaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm
import json
from ale.drivers.mro_drivers import MroCtxPds3LabelNaifSpiceDriver



In [229]:
from ale import spice_root 
from glob import glob
from os import path
from datetime import datetime 

mission = "mro"
instrument_id = "HiRise"
utc_time = datetime(year=2015, month=3, day=10)

def read_pref(path):
    with open(path) as f:
        preftext = f.read().replace('EndGroup', 'End_Group').replace("EndObject", "End_Object")
        pvlprefs = pvl.loads(preftext)
    return pvlprefs


mission_dir = path.join(spice_root, "mro")
kernel_dir = path.join(spice_root, "mro", "kernels")

kernel_types = [ name for name in os.listdir(kernel_dir) if os.path.isdir(os.path.join(kernel_dir, name)) ]

db_files = []
for typ in kernel_types:
    files = glob(path.join(kernel_dir, typ, "*.db"))
    if files:
        db_files.append(read_pref(sorted(files)[-1]))
    else:
        db_files.append(None)

        
db_files = dict(zip(kernel_types, db_files))

In [230]:
db_files['ck']

PVLModule([
  ('SpacecraftPointing',
   {'Dependencies': {'LeapsecondKernel': '$base/kernels/lsk/naif0012.tls',
                     'SpacecraftClockKernel': '$mro/kernels/sclk/MRO_SCLKSCET.00082.65536.tsc'},
    'RunTime': datetime.datetime(2019, 9, 13, 0, 18, 46),
    'Selection': {'File': '$mro/kernels/ck/mro_sc_psp_060911_060915p.bc',
                  'Time': ['2006 SEP 11 18:11:01.857682 TDB',
                           '2006 SEP 15 14:00:56.936282 TDB'],
                  'Type': 'Predicted'},
    'Selection': {'File': '$mro/kernels/ck/mro_sc_cru_050812_050817.bc',
                  'Time': ['2005 AUG 10 15:33:08.983036 TDB',
                           '2005 AUG 19 00:01:04.109429 TDB'],
                  'Type': 'Reconstructed'},
    'Selection': {'File': '$mro/kernels/ck/mro_sc_cru_050818_050824.bc',
                  'Time': ['2005 AUG 18 00:01:04.810619 TDB',
                           '2005 AUG 26 00:01:04.064393 TDB'],
                  'Type': 'Reconstructed'},
    'Selec

datetime.datetime(2005, 8, 19, 0, 1, 4, 109429)

In [236]:
kernels = {}

def parse_ik_db(pvlobj):
    if not pvlobj:
        return 
    
#     print(pvlobj.get("RunTime", None))
#     print(pvlobj.get("Dependencies", None))
    
#     print(pvlobj)
    run_time = None
    dependencies = None
    kernel = None
    quality = None
    
    for obj in pvlobj:
        if obj[0] == "Selection":
            obj = obj[1]
            match = obj.get("Match")
            file = obj.get("File")
            time = obj.get("Time")
            
            if time: 
                isis_time_format = '%Y %b %d %H:%M:%S.%f TDB'
                time = (datetime.strptime(time[0], isis_time_format), 
                                   datetime.strptime(time[1], isis_time_format))
                    
            if not file:
                raise Exception(f"No File found in {pvlobj}")
            elif isinstance(file, list):
                file = file[1]
            
            # acceptance for text kernels
            if not time and (not match or instrument_id.lower() == match[2].lower()):
                kernel = file
                break
            #acceptence for position/orientation kernels
            elif time and utc_time >= time[0] and utc_time <= time[1]:
                kernel = file
                break
            
        elif obj[0] == "Dependencies":
            dependencies = obj[1]
            
        elif obj[0] == "RunTime":
            run_time = obj[1]
    return kernel
    
kernels['fk'] = parse_ik_db(db_files['fk']["Frame"])
kernels['ik'] = parse_ik_db(db_files['ik']["Instrument"])
kernels['sclk'] = parse_ik_db(db_files['sclk']['SpacecraftClock'])
kernels['iak'] = parse_ik_db(db_files['iak']['InstrumentAddendum'])
kernels['ck'] = parse_ik_db(db_files['ck']['SpacecraftPointing'])
kernels['spk'] = parse_ik_db(db_files['spk']['SpacecraftPosition'])

In [237]:
kernels

{'fk': 'kernels/fk/mro_v??.tf',
 'ik': 'kernels/ik/mro_hirise_v??.ti',
 'sclk': 'kernels/sclk/MRO_SCLKSCET.?????.65536.tsc',
 'iak': 'kernels/iak/hiriseAddendum???.ti',
 'ck': '$mro/kernels/ck/mro_sc_psp_150303_150309.bc',
 'spk': '$mro/kernels/spk/mro_psp34.bsp'}

In [77]:
db_files["iak"]

PVLModule([
  ('InstrumentAddendum',
   {'Selection': {'File': ['mro', 'kernels/iak/hiriseAddendum???.ti'],
                  'Match': ['Instrument', 'InstrumentId', 'HIRISE']},
    'Selection': {'File': ['mro', 'kernels/iak/mroctxAddendum???.ti'],
                  'Match': ['Instrument', 'InstrumentId', 'CTX']},
    'Selection': {'File': ['mro', 'kernels/iak/marciAddendum???.ti'],
                  'Match': ['Instrument', 'InstrumentId', 'Marci']},
    'Selection': {'File': ['mro', 'kernels/iak/crismAddendum???.ti'],
                  'Match': ['Instrument', 'InstrumentId', 'CRISM']}})
])

In [28]:
import pvl
read_pref(db_files['ik'])

PVLModule([
  ('Instrument',
   {'Selection': {'File': ['mro', 'kernels/ik/mro_marci_v??.ti'],
                  'Match': ['Instrument', 'InstrumentId', 'Marci']},
    'Selection': {'File': ['mro', 'kernels/ik/mro_hirise_v??.ti'],
                  'Match': ['Instrument', 'InstrumentId', 'HIRISE']},
    'Selection': {'File': ['mro', 'kernels/ik/mro_crism_v??.ti'],
                  'Match': ['Instrument', 'InstrumentId', 'CRISM']}})
])

In [33]:
read_pref(db_files[''])

PVLModule([
  ('SpacecraftPosition',
   {'Dependencies': {'LeapsecondKernel': '$base/kernels/lsk/naif0012.tls'},
    'RunTime': datetime.datetime(2019, 9, 13, 0, 19, 34),
    'Selection': {'File': '$mro/kernels/spk/mro_psp.bsp',
                  'Time': ['2019 JUL 01 00:01:10.000000 TDB',
                           '2019 NOV 11 14:45:00.000000 TDB'],
                  'Type': 'Predicted'},
    'Selection': {'File': '$mro/kernels/spk/mro_cruise.bsp',
                  'Time': ['2005 AUG 12 12:41:56.000000 TDB',
                           '2006 MAR 10 22:10:10.000000 TDB'],
                  'Type': 'Reconstructed'},
    'Selection': {'File': '$mro/kernels/spk/mro_ab.bsp',
                  'Time': ['2006 MAR 10 22:05:10.000000 TDB',
                           '2006 SEP 12 06:45:00.000000 TDB'],
                  'Type': 'Reconstructed'},
    'Selection': {'File': '$mro/kernels/spk/mro_psp1.bsp',
                  'Time': ['2006 SEP 12 06:40:00.000000 TDB',
                           '2

In [9]:
kernel_types

['ck', 'sclk', 'ik', 'iak', 'spk', 'mk', 'fk']

In [57]:
%%time

# change to desired PDS3 image path 
fileName = '../tests/pytests/data/EN1072174528M/EN1072174528M_pds3.lbl'


from ale import util
import time 

class Spice(dict):
    def __getitem__(self, key):
        if key not in self.keys():
            self.__setitem__(key, util.duckpool(key))
        return super().__getitem__(key)
    
    def query(self, query):
        return util.query_kernel_pool(query)


# metakernels are furnished when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 
with MroCtxPds3LabelNaifSpiceDriver(fileName) as driver:
    # pass driver instance into formatter function
#     usgscsmString = to_usgscsm(driver)
    pass

SpiceyError: 
================================================================================

Toolkit version: N0066

SPICE(NOSUCHFILE) --

The first file '/data/spice/mro-m-spice-6-v1.0/mrosp_1000/data//lsk/naif0011.tls' specified by KERNELS_TO_LOAD in the file /scratch/spice/mro-m-spice-6-v1.0/mrosp_1000/extras/mk/mro_2015_v05.tm could not be located.

furnsh_c --> FURNSH --> ZZLDKER

================================================================================

In [16]:
from glob import glob 

infofil = "/data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/data/ik/ikinfo.txt"

In [17]:
# cat /data/spice/naif.jpl.nasa.gov/mess-e_v_h-spice-6-v1.0/messsp_1000/data/ck/ckinfo.txt

In [2]:
import re

template = re.compile("((msgr_)(([a-zA-Z0-9_]*(((<[yYmMdDn]+>)+)|\.)+)+)+)")

template = re.compile("(([a-zA-Z]+_)(([a-zA-Z0-9_]*(((<[yYmMdDn]+>)+)|\.)+)+)+)")
file_templates = [m.group(1) for m in template.finditer(open(infofil).read().lower())]
file_templates

NameError: name 'infofil' is not defined

In [3]:
matched = []

for i,t in enumerate(file_templates):
    t = os.path.splitext(t)[0]
    
    while t.find('<') != -1:
        ts = t[t.find('<'):t.find('>')+1]
        t = t.replace(ts, '.'*(len(ts)-2))
    
    file_templates[i] = t
#     base = '/data/spice/naif.jpl.nasa.gov/mess-e_v_h-spice-6-v1.0/messsp_1000/data/ck/'
    
#     for k in kernels:
#         i = os.path.splitext(os.path.basename(k))[0]
#         i = re.fullmatch(t, i)
#         if i:
#             matched.append(k)

# len(set(matched)), len(set(kernels))

NameError: name 'file_templates' is not defined

In [183]:
import spiceypy as spice

In [173]:
import datetime 
from collections import OrderedDict
import re 

template = {
    "mro" : {
        "predict" : ["mro_sc_psp_%y%m%d_%y%m%dp"],
        "recon" : ["mro_sc_cru_%y%m%d_%y%m%d",
                     "mro_sc_ab_%y%m%d_%y%m%d",
                     "mro_sc_psp_%y%m%d_%y%m%d",
                     "mro_sc_psp_%y%m%d_%y%m%d_v2"]
    }
}

file = "mro_sc_psp_080311_080317.bc"
components = os.path.splitext(file)[0].split('_')

temp = template["mro"]["recon"][2]

def match_template(template, file):
    for c,t in zip(components, temp.split("_")):
        res = []
        if "%" in t:
            time = datetime.datetime.strptime(c, t)
            new_c = time.strftime(t)
            is_match = re.fullmatch(new_c, c)
            assert is_match
        else:
            assert c == t


# template = OrderedDict({
#     "mission" : "*",
#     "spacecract" : "*",
#     "phase" : "*",
#     "start_time" : "%y%m%d",
#     "stop_time" : "%y%m%d"
# })





# metadata = {}

# for i,key in enumerate(template.keys()):
#     if isinstance(template[key], str):
#         try:
#             assert components[i] == template[key]
#             metadata[key] = components[i]
#         except:
#             time = datetime.datetime.strptime(components[i], template[key])
#             metadata[key] = time

#     elif isinstance(template[key], list):
#         assert components[i] in template[key]


2008-03-11 00:00:00
080311
2008-03-17 00:00:00
080317


In [161]:
t = template["reconstructed"][2].split('_')

datetime.datetime.strptime(file, )


KeyError: 'reconstructed'

In [145]:
metadata

{'mission': 'mro',
 'spacecract': 'sc',
 'start_time': datetime.datetime(2008, 3, 11, 0, 0),
 'stop_time': datetime.datetime(2008, 3, 17, 0, 0)}

In [122]:

from ale import spice_root
import pvl
from datetime import datetime
from glob import glob
import re 

print(spice_root)

mro_template = {
    
}


def insensitive_glob(pattern):
    def either(c):
        return '[%s%s]' % (c.lower(), c.upper()) if c.isalpha() else c
    return glob(''.join(map(either, pattern)))


cache = {}

def query_kernels(spice_root=spice_root, missions='all', times='all', versions='latest'):
    template = os.path.join(spice_root, f'{missions}*', '*', '*', '**', '*.lbl')
    labels = insensitive_glob(template)
    print(open(labels[1], 'rb').read())
    result = cache.get(tuple(labels), {})
    if result:
        return result
    
    print(len(labels))

    for l in labels:
        lines = []
        i = 0
        with open(l, 'r') as f:
            for line in f:
                i+=1
                if any(rec in line for rec in ['START_TIME', 'STOP_TIME']):
                    lines.append(line)
                if len(lines) >= 2:
                    break
        metadata = pvl.loads("\n".join(lines))
        kernel_file = metadata.get("^SPICE_KERNEL", None)
        kernel_type = metadata.get("KERNEL_TYPE_ID", None)
        if kernel_type:
            kernel_type = kernel_type.lower()
        
        start_epoch_time = metadata.get("START_TIME", None)
        stop_epoch_time = metadata.get("STOP_TIME", None)
        if isinstance(start_epoch_time, datetime):
            start_epoch_time = int(start_epoch_time.timestamp()*1000)
        else:
            start_epoch_time = None
            
        if isinstance(stop_epoch_time, datetime):
            stop_epoch_time = int(stop_epoch_time.timestamp()*1000)
        else:
            stop_epoch_time = None
        
        try:
            epoch_range = range(start_epoch_time, stop_epoch_time)
        except:
            epoch_range = None
        
        if kernel_type not in result.keys():
            result[kernel_type] = []
        
        result[kernel_type].append({
            'kernel' : kernel_file,
            'epoch_range' : epoch_range,
            'path' : l,
            'type' : kernel_type,
            'start_utc_time' : metadata.get("START_TIME", None),
            'stop_utc_time' : metadata.get("STOP_TIME", None),
            'instrument' : kernel_file.split('_')[1].lower() if kernel_type == "ik" else None
        })
        
#     cache[tuple(labels)] = result
    return result 

/scratch/spice


In [116]:
open(labels[1]).read()

NameError: name 'labels' is not defined

In [123]:
%%time
kernels = query_kernels(missions='mro')

b'PDS_VERSION_ID               = PDS3\r\nRECORD_TYPE                  = STREAM\r\nRECORD_BYTES                 = "N/A"\r\n^SPICE_KERNEL                = "mro_v14.tf"\r\nMISSION_NAME                 = "MARS RECONNAISSANCE ORBITER"\r\nSPACECRAFT_NAME              = "MARS RECONNAISSANCE ORBITER"\r\nDATA_SET_ID                  = "MRO-M-SPICE-6-V1.0"\r\nKERNEL_TYPE_ID               = FK\r\nPRODUCT_ID                   = "mro_v14.tf"\r\nPRODUCT_CREATION_TIME        = 2009-02-27T12:08:41\r\nPRODUCER_ID                  = "NAIF/JPL"\r\nMISSION_PHASE_NAME           = "N/A"\r\nPRODUCT_VERSION_TYPE         = ACTUAL\r\nPLATFORM_OR_MOUNTING_NAME    = "N/A"\r\nSTART_TIME                   = "N/A"\r\nSTOP_TIME                    = "N/A"\r\nSPACECRAFT_CLOCK_START_COUNT = "N/A"\r\nSPACECRAFT_CLOCK_STOP_COUNT  = "N/A"\r\nTARGET_NAME                  = MARS\r\nINSTRUMENT_NAME              = "N/A"\r\nNAIF_INSTRUMENT_ID           = "N/A"\r\nSOURCE_PRODUCT_ID            = "N/A"\r\nNOTE                     

In [111]:
kernels

{None: [{'kernel': None,
   'epoch_range': None,
   'path': '/scratch/spice/mro-m-spice-6-v1.0/mrosp_1000/data/fk/mro_v10.lbl',
   'type': None,
   'start_utc_time': None,
   'stop_utc_time': None,
   'instrument': None},
  {'kernel': None,
   'epoch_range': None,
   'path': '/scratch/spice/mro-m-spice-6-v1.0/mrosp_1000/data/fk/mro_v14.lbl',
   'type': None,
   'start_utc_time': None,
   'stop_utc_time': None,
   'instrument': None},
  {'kernel': None,
   'epoch_range': None,
   'path': '/scratch/spice/mro-m-spice-6-v1.0/mrosp_1000/data/fk/mro_v11.lbl',
   'type': None,
   'start_utc_time': None,
   'stop_utc_time': None,
   'instrument': None},
  {'kernel': None,
   'epoch_range': None,
   'path': '/scratch/spice/mro-m-spice-6-v1.0/mrosp_1000/data/fk/mro_v15.lbl',
   'type': None,
   'start_utc_time': None,
   'stop_utc_time': None,
   'instrument': None},
  {'kernel': None,
   'epoch_range': None,
   'path': '/scratch/spice/mro-m-spice-6-v1.0/mrosp_1000/data/pck/pck00008.lbl',
   'ty

In [6]:
%%time

kernels = []
for l in labels:
    lines = []
    with open(l) as f:
        for line in f:
            if "START_TIME" in line or "STOP_TIME" in line or 'SPICE_KERNEL' in line:
                lines.append(line)
            if len(lines) == 3:
                break
                
    metadata = pvl.loads("\n".join(lines))
    kernel_file = metadata["^SPICE_KERNEL"]
    compenents = tuple(kernel_file.split('_'))
    
    kernels.append({
        'kernel' : kernel_file,
        'date_range' : range(int(metadata['START_TIME'].timestamp()*1000), int(metadata['STOP_TIME'].timestamp()*1000)),
        'path' : l
    })

NameError: name 'labels' is not defined

In [63]:
kernels['ck']

[{'kernel': 'mro_sc_psp_080715_080721.bc',
  'epoch_range': range(1216105200162, 1216710059847),
  'path': '/scratch/spice/mro-m-spice-6-v1.0/mrosp_1000/data/ck/mro_sc_psp_080715_080721.lbl',
  'type': 'ck',
  'start_utc_time': datetime.datetime(2008, 7, 15, 0, 0, 0, 162000),
  'stop_utc_time': datetime.datetime(2008, 7, 22, 0, 0, 59, 847000),
  'instrument': None},
 {'kernel': 'mro_sa_psp_100727_100802.bc',
  'epoch_range': range(1280214007417, 1280819099387),
  'path': '/scratch/spice/mro-m-spice-6-v1.0/mrosp_1000/data/ck/mro_sa_psp_100727_100802.lbl',
  'type': 'ck',
  'start_utc_time': datetime.datetime(2010, 7, 27, 0, 0, 7, 417000),
  'stop_utc_time': datetime.datetime(2010, 8, 3, 0, 4, 59, 387000),
  'instrument': None},
 {'kernel': 'mro_hga_psp_101116_101122.bc',
  'epoch_range': range(1289890809157, 1290495831253),
  'path': '/scratch/spice/mro-m-spice-6-v1.0/mrosp_1000/data/ck/mro_hga_psp_101116_101122.lbl',
  'type': 'ck',
  'start_utc_time': datetime.datetime(2010, 11, 16, 0

In [15]:
import pvl
dict(pvl.load(labels[100]))

{'PDS_VERSION_ID': 'PDS3',
 'RECORD_TYPE': 'FIXED_LENGTH',
 'RECORD_BYTES': 1024,
 '^SPICE_KERNEL': 'msgr_0711_v02.bc',
 'MISSION_NAME': 'MESSENGER',
 'SPACECRAFT_NAME': 'MESSENGER',
 'DATA_SET_ID': 'MESS-E/V/H-SPICE-6-V1.0',
 'KERNEL_TYPE_ID': 'CK',
 'PRODUCT_ID': 'msgr_0711_v02.bc',
 'PRODUCT_CREATION_TIME': datetime.datetime(2009, 3, 14, 21, 31),
 'PRODUCER_ID': 'JHUAPL',
 'MISSION_PHASE_NAME': 'MERCURY 1 CRUISE',
 'PRODUCT_VERSION_TYPE': 'ACTUAL',
 'PLATFORM_OR_MOUNTING_NAME': 'N/A',
 'START_TIME': datetime.datetime(2007, 10, 31, 23, 59, 38, 402000),
 'STOP_TIME': datetime.datetime(2007, 12, 1, 0, 2, 41, 578000),
 'SPACECRAFT_CLOCK_START_COUNT': '1/102362533:000375',
 'SPACECRAFT_CLOCK_STOP_COUNT': '1/104954733:000150',
 'TARGET_NAME': {'EARTH', 'MERCURY', 'VENUS'},
 'INSTRUMENT_NAME': 'N/A',
 'NAIF_INSTRUMENT_ID': {-236002, -236001, -236000},
 'SOURCE_PRODUCT_ID': 'N/A',
 'NOTE': 'See comments in the file for details',
 'SPICE_KERNEL': PVLObject([
   ('INTERCHANGE_FORMAT', 'BINARY

In [22]:
labels[0]

'/scratch/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/data/ck/msgr_mdis_sc_040812_070610.lbl'

In [25]:
pvl.load("/scratch/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/data/ck/msgr_mdis_sc_040812_070610.bc")

ParseError: Unexpected token b'\x00' (expected b'='): line 1 column 10 (char 9)

### Write ISD to disk 

ALE formatter functions generally return bytes or a string that can be written out to disk. ALE's USGSCSM formatter function returns a JSON encoded string that can be written out using any JSON library. 

USGSCSM requires the ISD to be colocated with the image file with a `.json` extension in place of the image extension.

In [4]:
# load the json encoded string ISD
usgscsm_dict = json.loads(usgscsmString)

# strip the image file extension and append .json 
jsonFile = os.path.splitext(fileName)[0] + '.json'

# write to disk 
with open(jsonFile, 'w') as fp:
    json.dump(usgscsm_dict, fp)